「PyTorch入門  4. モデル構築」
===============================================================
【原題】BUILD THE NEURAL NETWORK

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/)


【元URL】https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月18日

【チュトーリアル概要】

本チュートリアルでは、PyTorchでニューラルネットワークのモデルを構築する方法について解説します

---


ニューラルネットワークモデルの作り方
===================

ニューラルネットワークは、レイヤー（もしくはモジュール）と呼ばれるデータ操作の固まりで構成されています。

[`torch.nn`](https://pytorch.org/docs/stable/nn.html)で用意されているクラス、関数は、独自のニューラルネットワークを構築するために必要な要素を網羅しています。



PyTorchの全てのモジュールは、[`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承しています。


そしてニューラルネットワークは、モジュール自体が他のモジュール（レイヤー）から構成されています。

<br>

この入れ子構造により、複雑なアーキテクチャを容易に構築・管理することができます。

以下で、FashionMNIST datasetの画像データをクラス分類するネットワークモデルを構築します。




In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

---

訓練に使用するデバイス
-----------------------
可能であれば高速に訓練を行うためにも、GPU等を利用したいところです。

GPUが使用可能かどうかは、[`torch.cuda`](https://pytorch.org/docs/stable/notes/cuda.html)の結果から判断できます。

GPUが使用できない場合は、CPUで訓練を行います。



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


---



クラスの定義
-------------------------
``nn.Module``を継承し、独自のネットワークモデルを定義し、その後ネットワークのレイヤーを ``__init__``で初期化します。

``nn.Module`` を継承した全モジュールは、入力データの順伝搬関数である``forward``関数を持ちます。





In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

``NeuralNetwork``クラスのインスタンスを作成し、変数``device``上に移動させます。

以下でネットワークの構造を出力し確認します。



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを利用するには、入力データを与える必要があります。

入力データをモデルに投入すると、``forward``関数で処理されるとともに、いくつかの[`background operations`](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)が実行されます。

そのため、 ``model.forward()`` と記載して入力データを処理しないように注意してください。



今回のモデルに入力を与えると、各クラスの生の予測値を含む10次元のテンソルが返されます。

``nn.Softmax``モジュールにこの出力結果を与えることで、入力データが各クラスに属する確率の予測値を求めることができます。





In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


--------------




モデルレイヤー
-------------------------

FashionMNISTモデルを各レイヤーレベルで確認しましょう。

サイズ28x28の3枚の画像からなるミニバッチのサンプルを用意し、このミニバッチをネットワークに入力し、各処理による変化を確認していきます。



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**

[`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)レイヤーで、2次元（28x28）の画像を、1次元の784ピクセルの値へと変換します。

ミニバッチの0次元目は、サンプル番号を示す次元で、この次元は`nn.Flatten`を通しても変化しません（1次元目以降がFlattenされます）。



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear**


[`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は、線形変換を施します。

`linear layer`は重みとバイアスのパラメータを保持しています。





In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**

非線形な活性化関数は、ニューラルネットワークの入力と出力の間にある、複雑な関係性を表現するために重要な要素です。

これらの活性化関数は線形変換のあとに、非線形性を加え、ニューラルネットワークの表現力を向上させる役割をします。

<br>

今回のモデルでは[`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)をlinear layerの間に用意しました。

なお、非線形活性化関数には他にも様々な種類があります。



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1060,  0.0821,  0.3766, -0.1524,  0.2117, -0.2510, -0.4499, -0.0075,
         -0.1151,  0.5875,  0.7538,  0.1273, -0.2190, -0.0395, -0.1491, -0.0975,
         -0.3546,  0.1713, -0.1335,  0.1907],
        [-0.1643,  0.3026,  0.5124,  0.0795,  0.1503, -0.2320, -0.2538, -0.0823,
         -0.0749,  0.5273,  0.0387, -0.1703, -0.1171,  0.0012, -0.1024,  0.1369,
         -0.0781,  0.1847, -0.0084,  0.1532],
        [ 0.2198,  0.2919,  0.5265, -0.0613,  0.2676, -0.3340, -0.4021,  0.0221,
         -0.0130,  0.3402,  0.8478,  0.1457, -0.3019, -0.2789, -0.1507, -0.2266,
         -0.0942,  0.0686, -0.1774, -0.0457]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1060, 0.0821, 0.3766, 0.0000, 0.2117, 0.0000, 0.0000, 0.0000, 0.0000,
         0.5875, 0.7538, 0.1273, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1713,
         0.0000, 0.1907],
        [0.0000, 0.3026, 0.5124, 0.0795, 0.1503, 0.0000, 0.0000, 0.0000, 0.0000,
         0.5273, 0.0387, 0.0000, 0.0000, 0.0012, 0.00

**nn.Sequential**

[``nn.Sequential``](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、モジュールを順番に格納する箱のような要素です。

入力データは``nn.Sequential``に定義された順番に各モジュールを伝搬します。

<br>

例えば以下の実装例のように、``seq_modules``と名付けた、複数のモジュールを束ねたモジュールを簡単に構築できます。




In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

**nn.Softmax**

ニューラルネットワークの最後のlinear layerは`logits` [- ∞, ∞] を出力します。


この`logits`は[`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)モジュールへと渡されます。


その結果、採取的な値は[0, 1]の範囲となり、これは各クラスである確率を示します。


``dim``パラメータは次元を示しており、`dim=1`の次元で和を求めると確率の総和なので1になります。




In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

---




モデルパラメータ
-------------------------

ニューラルネットワークを構成する多くのモジュールは、おのおのパラメータを保持しています。

例えば、重みやバイアスです。これらの値が訓練時に最適化されます。




``nn.Module`` を継承することで、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡でき、``parameters()`` や ``named_parameters()`` メソッドを使って、モデルの各レイヤーのすべてのパラメータにアクセスできるようになります。

以下にfor文を用いて、各パラメータを処理し、そのサイズと値を表示します。



In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0288, -0.0291,  0.0066,  ..., -0.0050,  0.0132,  0.0107],
        [ 0.0082, -0.0020,  0.0172,  ..., -0.0061, -0.0294, -0.0067]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0002, -0.0032], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0110,  0.0279, -0.0246,  ...,  0.0378,  0.0081,  0.0079],
        [-0.0286,  0.0279, -0.0360,  ..., -0.0089,  0.0133, -0.0261]],
       device='cu

--------------




さらなる詳細
--------------
以下のページも参考ください。

- [`torch.nn API`](https://pytorch.org/docs/stable/nn.html)



以上。